## Interactive tactics board with pitch control

Click and drag the dots to change player positions. Click and drag the arrow heads to change player velocities. Interactive visualisation based on [bqplot](https://github.com/bqplot/bqplot). Pitch control model due to Spearman (Beyond Expected Goals, Sloan Sports Conference, 2018). Pitch control code slightly modified from [Laurie Shaw's implementation](https://github.com/Friends-of-Tracking-Data-FoTD/LaurieOnTracking/blob/master/Metrica_PitchControl.py). All of the code is available [here](https://github.com/anenglishgoat/InteractivePitchControl).

In [1]:
import numpy as np
import ipywidgets
from bqplot import *
import bqplot.pyplot as plt
import matplotlib.pyplot as plt2

def get_contours(Z,x,y,n_contour = 10):
    levels = np.linspace(0,1,n_contour)
    cont = plt2.contourf(xx,yy,Z,levels = np.linspace(0,1,n_contour+1),nchunk=10)
    plt2.close()
    contours = [cs.get_paths() for cs in cont.collections]
    levels = [np.repeat(levels[i],len(contours[i])) for i in range(n_contour)]
    contours = [item for sublist in contours for item in sublist]
    contours = [c.vertices for c in contours]
    contours_x = [list(c[:,0])  for c in contours]
    contours_y = [list(c[:,1]) for c in contours]
    return contours_x,contours_y, levels

def get_pitch_control_proportion(Z):
    pc = np.sum(Z) / np.prod(Z.shape) * 100
    return '%s' % float('%.3g' % pc) + '%'

def get_pitch_control(home_pos,
                      away_pos,
                      home_v,
                      away_v,
                      ball_pos,
                      target_position):
    
    average_ball_speed = 15.
    reaction_time = 0.7
    max_player_speed = 5.
    time_to_control = 3. * np.log(10.) * (np.sqrt(3) * 0.45 + 1/4.3)    

    max_int_time = np.array([0.,10.])
    
    ball_travel_time = np.linalg.norm(xxyy - np.array([50.,50.]),axis = 2)/average_ball_speed
    
    # first get arrival time of 'nearest' attacking player (nearest also dependent on current velocity)
    
    r_reaction_home = home_pos + reaction_time * home_v
    r_reaction_away = away_pos + reaction_time * away_v
    
    tti_home = reaction_time + np.linalg.norm(target_position[None,:,:,:] - r_reaction_home[:,None,None,:],axis = 3) / max_player_speed
    tti_away = reaction_time + np.linalg.norm(target_position[None,:,:,:] - r_reaction_away[:,None,None,:],axis = 3) / max_player_speed
    tti = np.concatenate([tti_home,tti_away]).flatten('F')
    
    y = np.zeros_like(tti)

    for tt in np.arange(0,10,0.02):
        yy = np.reshape(y,(22,-1),order = 'F')
        sums = np.sum(yy,axis = 0)
        sumy = np.repeat(sums,22)
        y += 0.02 * 4.3 * (1 - sumy) * 1/(1. + np.exp(-np.pi/np.sqrt(3.)/0.45 * (tt-tti) ) )
    
    return np.sum(np.reshape(y,(22,50,50),'F')[:11],axis=0)

In [2]:
x_start_home = np.array([10,30,30,30,30,55,55,55,55,70,70])
y_start_home = np.array([50,10,35,60,90,10,35,60,90,35,60])
x_start_away = 100 - x_start_home
y_start_away = 100 - y_start_home

In [3]:
fig = plt.figure()
fig.min_aspect_ratio = 105/74.8
fig.max_aspect_ratio = 115.5/74.8
plt.plot([[0,0],
         [0,100],
         [100,100],
         [100,0]],
         
         [[0,100],
         [0,0],
         [0,100],
         [100,100]],         
         colors=['black'],stroke_width = 1,
         axes_options={'x': {'visible': False}, 'y': {'visible': False}})
plt.scatter([25,35,50,70],
            [105,105,105,105],
           colors = ['white'],
           stroke = 'orange')
plt.plot([[25,26],[35,37],[50,57],[70,82]],
         [[105,105],[105,105],[105,105],[105,105]],
         colors = ['black'],
         stroke_width = 1.)
plt.scatter([26,37,57,82],
            [105,105,105,105],
            colors = ['black'],
            default_size = 4,
            stroke_width = 1.,
            marker = 'triangle-up',
            scales={'rotation': LinearScale(min=0, max=180)},
            rotation = [90.] * 4)

plt.label(['Velocity guide:','Walk','Jog','Sprint','Usain Bolt'],
            x=[1,22.5,33,49,69],
            y=[108,110,110,110,110],
           colors = ['black'],
         default_size = 12)
fig

Figure(axes=[Axis(scale=LinearScale(), visible=False), Axis(orientation='vertical', scale=LinearScale(), visib…

In [5]:
xx = np.linspace(0,100,50)
yy = np.linspace(0,100,50)
XX,YY = np.meshgrid(xx,yy)
xxyy = np.dstack((XX,YY))
targets = xxyy * np.array([105/100,68/100])

cm=plt2.get_cmap('coolwarm')

home_pos = np.c_[x_start_home,y_start_home] * np.array([105/100,68/100])
away_pos = np.c_[x_start_away,y_start_away] * np.array([105/100,68/100])
home_v = np.c_[np.ones_like(x_start_home)*5,np.zeros_like(x_start_home)] * np.array([105/100,68/100])
away_v = np.c_[np.ones_like(x_start_away)*-5,np.zeros_like(x_start_away)] * np.array([105/100,68/100])
ball_pos = np.array([50.,50.]) * np.array([105/100,68/100])


X_start = get_pitch_control(home_pos,
                            away_pos,
                            home_v,
                            away_v,
                            ball_pos,
                            targets)

contours_x,contours_y,levels = get_contours(X_start,xx,yy)

cols = (cm(np.concatenate(levels))[:,:3]*255) * 0.8 + 25.5
hmap = plt.plot([],[],
                 fill_colors=['rgb(' + str(int(c[0])) + ',' + str(int(c[1])) + ',' + str(int(c[2])) + ')' for c in cols],
                 fill='inside',
                 axes_options={'x': {'visible': False}, 'y': {'visible': False}},
                 stroke_width=0,
                 close_path=False,
                 display_legend=False,
               fill_opacities = [1.]*len(contours_x))

hmap.interpolation = 'linear'
hmap.x = contours_x
hmap.y = contours_y
plt.ylim(0,110)
plt.xlim(0,100)

scat_home = plt.scatter(x_start_home, y_start_home, colors = ['white'],stroke='orange', enable_move=True)
scat_away = plt.scatter(x_start_away, y_start_away, colors=['white'],stroke='blue', enable_move=True)
scat_ball = plt.scatter([50.], [50.], colors=['white'], enable_move=True,stroke='black')

scat_v_home = plt.scatter(x_start_home+5, y_start_home, colors=['black'], enable_move=True,
                          marker = 'triangle-up',
                         default_size = 4,
                          scales={'rotation': LinearScale(min=0, max=180)},
                         rotation = [90.]*len(scat_home.x))
scat_v_away = plt.scatter(x_start_away-5, y_start_away, colors=['black'], enable_move=True,
                          marker='triangle-up',
                         default_size = 4,
                          scales={'rotation': LinearScale(min=0, max=180)},
                         rotation = [-90.]*len(scat_home.x))

lines_v_home = plt.plot(np.vstack([x_start_home,x_start_home+5]).T,
                        np.vstack([y_start_home,y_start_home]).T,
                        colors = ['black'],
                       stroke_width = 1.)

lines_v_away = plt.plot(np.vstack([x_start_away,x_start_away-5]).T,
                        np.vstack([y_start_away,y_start_away]).T,
                        colors = ['black'],
                       stroke_width = 1.)

plt.plot([[50,50],
         [0,16.5/105*100],
         [0,16.5/105*100],
         [100,100 - 16.5/105*100],
         [100,100 - 16.5/105*100],
         [16.5/105*100,16.5/105*100],
         [100-16.5/105*100,100 - 16.5/105*100],
         [0,5.5/105*100],
         [0,5.5/105*100],
         [100,100 - 5.5/105*100],
         [100,100 - 5.5/105*100],
         [5.5/105*100,5.5/105*100],
         [100-5.5/105*100,100 - 5.5/105*100]],
         
         [[0,100],
         [50-20/68*100,50-20/68*100],
         [50+20/68*100,50+20/68*100],
         [50-20/68*100,50-20/68*100],
         [50+20/68*100,50+20/68*100],
         [50-20/68*100,50+20/68*100],
         [50-20/68*100,50+20/68*100],
         [50-9/68*100,50-9/68*100],
         [50+9/68*100,50+9/68*100],
         [50-9/68*100,50-9/68*100],
         [50+9/68*100,50+9/68*100],
         [50-9/68*100,50+9/68*100],
         [50-9/68*100,50+9/68*100]],
         
         colors=['black'],stroke_width = 1)

plt.plot(50 + 9.15/105*100 * np.sin(np.linspace(0,2*np.pi,500)),
         50 + 9.15/68*100 * np.cos(np.linspace(0,2*np.pi,500)),
         colors=['black'],stroke_width = 1)

v_home_x = np.ones(len(scat_home.x)) * 5
v_home_y = np.ones(len(scat_home.x)) * 0

v_away_x = np.ones(len(scat_home.x)) * -5
v_away_y = np.ones(len(scat_home.x)) * 0

def update_hmap(change=None):
    global X_new
    with hmap.hold_sync():
        new_home_pos_x = scat_home.x
        new_home_pos_y = scat_home.y
        new_away_pos_x = scat_away.x
        new_away_pos_y = scat_away.y
        
        scat_v_home.y, scat_v_home.x = new_home_pos_y + v_home_y, new_home_pos_x + v_home_x
        scat_v_away.y, scat_v_away.x = new_away_pos_y + v_away_y, new_away_pos_x + v_away_x
        
        lines_v_home.x, lines_v_home.y = np.vstack([new_home_pos_x,new_home_pos_x + v_home_x]).T, np.vstack([new_home_pos_y,new_home_pos_y + v_home_y]).T
        lines_v_away.x, lines_v_away.y = np.vstack([new_away_pos_x,new_away_pos_x + v_away_x]).T, np.vstack([new_away_pos_y,new_away_pos_y + v_away_y]).T

        
        home_pos = np.c_[scat_home.x,scat_home.y] * np.array([105/100,68/100])
        away_pos = np.c_[scat_away.x,scat_away.y] * np.array([105/100,68/100])
        home_v = np.c_[v_home_x,v_home_y] * np.array([105/100,68/100])
        away_v = np.c_[v_away_x,v_away_y] * np.array([105/100,68/100])
        ball_pos = np.c_[scat_ball.x,scat_ball.y] * np.array([105/100,68/100])
        X_new = get_pitch_control(home_pos,
                                  away_pos,
                                  home_v,
                                  away_v,
                                  ball_pos,
                                  targets)
        
        contours_x,contours_y,levels = get_contours(X_new,xx,yy)
        cols= (cm(np.concatenate(levels))[:,:3]*255) * 0.8 + 25.5
        hmap.fill_colors=['rgb(' + str(int(c[0])) + ',' + str(int(c[1])) + ',' + str(int(c[2])) + ')' for c in cols]
        hmap.x = contours_x
        hmap.y = contours_y
                
update_hmap()
        
def update_v_markers_angle_home(change=None):
    #with scat_v_home.hold_sync():
        global v_home_x, v_home_y
        v_home_y = scat_v_home.y - scat_home.y
        v_home_x = scat_v_home.x - scat_home.x
        lines_v_home.x, lines_v_home.y = np.vstack([scat_home.x,scat_home.x + v_home_x]).T, np.vstack([scat_home.y,scat_home.y + v_home_y]).T
        scat_v_home.rotation = [np.degrees(np.arctan2(-yyy,xxx)) + 90. for yyy,xxx in zip(v_home_y,v_home_x)]
        
def update_v_markers_angle_away(change=None):
    #with scat_v_home.hold_sync():
        global v_away_x, v_away_y
        v_away_y = scat_v_away.y - scat_away.y
        v_away_x = scat_v_away.x - scat_away.x
        lines_v_away.x, lines_v_away.y = np.vstack([scat_away.x,scat_away.x + v_away_x]).T, np.vstack([scat_away.y,scat_away.y + v_away_y]).T
        scat_v_away.rotation = [np.degrees(np.arctan2(-yyy,xxx)) + 90. for yyy,xxx in zip(v_away_y,v_away_x)]
    
        

# update line on change of x or y of scatter
scat_v_home.observe(update_v_markers_angle_home, ['x','y'])
scat_v_away.observe(update_v_markers_angle_away, ['x','y'])
scat_home.observe(update_hmap, names=['x','y'])
scat_v_home.observe(update_hmap, ['x','y'])
scat_away.observe(update_hmap, names=['x','y'])
scat_v_away.observe(update_hmap, ['x','y'])
scat_ball.observe(update_hmap, names=['x','y'])

In [14]:
from IPython.display import display, clear_output
button = widgets.Button(description="Get red team control")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        print(get_pitch_control_proportion(X_new))

button.on_click(on_button_clicked)

Button(description='Get red team control', style=ButtonStyle())

Output()